In [30]:
import struct
import uuid
import binascii
import enum
import datetime as dt

In [31]:
packet = bytes.fromhex('')

In [32]:
class binaryPacket:
    def __init__(self, payload:bytes, packetClass:int, packetID:int, sourceUUID:uuid.UUID, destUUID:uuid.UUID):
        self._payload = payload
        self._class = packetClass
        self._id = packetID
        self._source = sourceUUID
        self._dest = destUUID
    
    def to_bytes(self):
        payloadLen = len(self._payload)
        header = struct.pack("<BB", 0xE4, 0xEB) + \
                             self._source.bytes + \
                             self._dest.bytes + \
                             struct.pack("<BBH", self._class, self._id, payloadLen)
        msg = header + self._payload
        cksum = binascii.crc_hqx(msg, 0xFFFF).to_bytes(2, "big")
        return msg + cksum
    
    def getClassIDCode(self):
        return (self._class << 8) | self._id
    
    def __str__(self):
        string = self.to_bytes().hex().upper()
        length = 4
        return '0x%s' % ' '.join(string[i:i + length] for i in range(0, len(string), length))
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, packet):
        return self.to_bytes() == packet.to_bytes()
    
    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        return binaryPacket(payload, pcls, pid, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))
    
    @classmethod
    def parseHeader(cls, packet:bytes):
        if binascii.crc_hqx(packet, 0xFFFF) != 0:
            raise RuntimeError("Checksum verification failed")
        
        if len(packet) < 0x28:
            raise RuntimeError("Packet too short!")
            
            
        s1, s2, pcls, pid, _ = struct.unpack("<BB16x16xBBH", packet[0:0x26])
        srcUUID = bytes(packet[0x0002:0x0012])
        destUUID = bytes(packet[0x0012:0x0022])
        if s1 != 0xE4 or s2 != 0xEB:
            raise RuntimeError("Not a packet!")
        payload = packet[0x26:-2]
        return srcUUID, destUUID, pcls, pid, payload
    
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return True

In [33]:
class E4E_Data_IMU(binaryPacket):
    __class = 0x04
    __id = 0x00
    __VERSION = 0x01

    def __init__(self, src:uuid.UUID, dest:uuid.UUID, accX, accY, accZ, gyroX, gyroY, gyroZ, magX, magY, magZ, timestamp:dt.datetime = None):
        self.acc = [accX, accY, accZ]
        self.gyro = [gyroX, gyroY, gyroZ]
        self.mag = [magX, magY, magZ]
        if timestamp is None:
            timestamp = dt.datetime.now()
        self.timestamp = timestamp
        
        
        payload = struct.pack("<BBQ3f3f3f",
                                   self.__VERSION,
                                   0x00,
                                   int(timestamp.timestamp() * 1e3),
                                   *self.acc,
                                   *self.gyro,
                                   *self.mag)
        super(E4E_Data_IMU, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id
    
    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, _, timestamp_ms, acc, gyro, mag = struct.unpack("<BBQ3f3f3f", payload)
        if ver != cls.__VERSION:
            return RuntimeError("Unknown packet version!")
        
        timestamp = dt.datetime.fromtimestamp(timestamp_ms / 1e3)
        return E4E_Data_IMU(src=uuid.UUID(bytes=srcUUID),
                           dest=uuid.UUID(bytes=destUUID),
                           accX = acc[0],
                           accY = acc[1],
                           accZ = acc[2],
                           gyroX = gyro[0],
                           gyroY = gyro[1],
                           gyroZ = gyro[2],
                           magX = mag[0],
                           magY = mag[1],
                           magZ = mag[2],
                           timestamp=timestamp)

In [34]:
class binaryPacketParser:
    class State(enum.Enum):
        FIND_SYNC1 = 0
        FIND_SYNC2 = 1
        HEADER = 2
        PAYLOAD = 3
        CKSUM = 4
        VALIDATE = 5
    packetMap = {
        
    }
    
    HEADER_LEN = 0x0026
    
    def __init__(self):
        self.__state = self.State.FIND_SYNC1
        self.__payloadLen = 0
        self.__buffer = None
    
    def parseByte(self, data:int):
        if self.__state is self.State.FIND_SYNC1:
            if data == 0xE4:
                self.__state = self.State.FIND_SYNC2
                self.__buffer = bytearray()
                self.__buffer.append(data)
            return None
        elif self.__state is self.State.FIND_SYNC2:
            if data == 0xEB:
                self.__state = self.State.HEADER
                self.__buffer.append(data)
            else:
                self.__state = self.State.FIND_SYNC1
            return None
        elif self.__state is self.State.HEADER:
            self.__buffer.append(data)
            if len(self.__buffer) == self.HEADER_LEN:
                self.__state = self.State.PAYLOAD
                self.__payloadLen, = struct.unpack('<H', self.__buffer[self.HEADER_LEN - 2:self.HEADER_LEN])
            return None
        elif self.__state is self.State.PAYLOAD:
            self.__buffer.append(data)
            if len(self.__buffer) == self.__payloadLen + self.HEADER_LEN:
                self.__state = self.State.CKSUM
            return None
        elif self.__state is self.State.CKSUM:
            self.__buffer.append(data)
            self.__state = self.State.VALIDATE
            return None
        elif self.__state is self.State.VALIDATE:
            self.__buffer.append(data)
            if binascii.crc_hqx(self.__buffer, 0xFFFF) != 0:
                raise RuntimeError("Checksum verificatin failed")
            packetID, = struct.unpack('>H', self.__buffer[0x0022:0x0024])
            self.__state = self.State.FIND_SYNC1
            if packetID not in self.packetMap:
                return binaryPacket.from_bytes(self.__buffer)
            else:
                return self.packetMap[packetID].from_bytes(self.__buffer)
    
    def parseBytes(self, data:bytes):
        packets = []
        for byte in data:
            retval = self.parseByte(byte)
            if retval is not None:
                packets.append(retval)
        return packets

In [35]:
imuPacket = E4E_Data_IMU(src=uuid.uuid4(),
                        dest=uuid.uuid4(),
                        accX=1.0,
                        accY=2.0,
                        accZ=3.0,
                        gyroX=4.0,
                        gyroY=5.0,
                        gyroZ=6.0,
                        magX=7.0,
                        magY=8.0,
                        magZ=9.0,
                        timestamp=dt.datetime(2020, 12, 24, 0, 13, 13))
print(imuPacket)
print(hex(len(imuPacket.to_bytes())))
parser = binaryPacketParser()
packets = parser.parseBytes(imuPacket.to_bytes())
print(packets)

0xE4EB 8A84 4A18 1759 4582 9EFC 6E9D 895D CB21 8535 DBA0 B677 4092 829E AC83 824E 32FE 0400 2E00 0100 A81D CF93 7601 0000 0000 803F 0000 0040 0000 4040 0000 8040 0000 A040 0000 C040 0000 E040 0000 0041 0000 1041 AB2E
0x56
[0xE4EB 8A84 4A18 1759 4582 9EFC 6E9D 895D CB21 8535 DBA0 B677 4092 829E AC83 824E 32FE 0400 2E00 0100 A81D CF93 7601 0000 0000 803F 0000 0040 0000 4040 0000 8040 0000 A040 0000 C040 0000 E040 0000 0041 0000 1041 AB2E]
